In [1]:
# This notebook is intented to test, some of the
# results validation of GPT2 model 

In [2]:
import tensorflow as tf
from tf_transformers.models.gpt2.gpt2_model import GPT2Model
from transformers import GPT2Tokenizer


In [4]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [5]:
# Check PT HF conversion
!rm -rf /tmp/tf_transformers_cache/gpt2/
model_name = 'gpt2'
model, conifg = GPT2Model.get_model(model_name=model_name, convert_fn_type='pt')

INFO:absl:Successful: Converted model using PT HF
INFO:absl:Successful: Saved model at /tmp/tf_transformers_cache/gpt2/ckpt-1
INFO:absl:Successful: Asserted and Converted `gpt2` from HF and saved it in cache folder /tmp/tf_transformers_cache/gpt2


In [5]:
# Check TF HF conversion
!rm -rf /tmp/tf_transformers_cache/gpt2/
model_name = 'gpt2'
model, conifg = GPT2Model.get_model(model_name=model_name, convert_fn_type='tf')

INFO:absl:Successful: Converted model using TF HF
INFO:absl:Successful: Saved model at /tmp/tf_transformers_cache/gpt2/ckpt-1
INFO:absl:Successful: Asserted and Converted `gpt2` from HF and saved it in cache folder /tmp/tf_transformers_cache/gpt2


In [9]:
# Text generation without caching

text = 'Sachin Tendulkar is one of the finest'
inputs_tf = tokenizer(text, return_tensors='tf')
inputs = {}
inputs['input_ids'] = inputs_tf['input_ids']

predictions_non_auto_regressive = []
predictions_prob_non_auto_regressive = []

for i in range(10):
    outputs = model(inputs)
    predicted_ids = tf.cast(tf.expand_dims(tf.argmax(outputs["last_token_logits"], axis=1), 1), tf.int32)
    inputs["input_ids"] = tf.concat([inputs["input_ids"], predicted_ids], axis=1)
    predictions_non_auto_regressive.append(predicted_ids)
    predictions_prob_non_auto_regressive.append(
        tf.expand_dims(tf.reduce_max(outputs["last_token_logits"], axis=1), 1)
    )
predictions_non_auto_regressive = tf.concat(predictions_non_auto_regressive, axis=1)
predictions_prob_non_auto_regressive = tf.concat(predictions_prob_non_auto_regressive, axis=1)


# Text generation with caching

model, conifg = GPT2Model.get_model(model_name=model_name, use_auto_regressive=True)
inputs_tf = tokenizer(text, return_tensors='tf')
inputs = {}
inputs['input_ids'] = inputs_tf['input_ids']

seq_length = tf.shape(input_ids)[1]
batch_size = tf.shape(input_ids)[0]

inputs["all_cache_key"] = tf.zeros((12, batch_size, 12, seq_length, 64))
inputs["all_cache_value"] = tf.zeros((12, batch_size, 12, seq_length, 64))
inputs["past_length"] = tf.zeros(shape=(1, batch_size), dtype=tf.int32)
predictions_auto_regressive = []
predictions_prob_auto_regressive = []

past_lengths = []
for i in range(10):
    outputs = model(inputs)
    predicted_ids = tf.cast(tf.expand_dims(tf.argmax(outputs["last_token_logits"], axis=1), 1), tf.int32)
    inputs["input_ids"] = predicted_ids
    inputs["all_cache_key"] = outputs["all_cache_key"]
    inputs["all_cache_value"] = outputs["all_cache_value"]
    inputs["past_length"] = outputs["past_length"]
    past_lengths.append(inputs["past_length"])
    predictions_auto_regressive.append(predicted_ids)
    predictions_prob_auto_regressive.append(
        tf.expand_dims(tf.reduce_max(outputs["last_token_logits"], axis=1), 1)
    )
predictions_auto_regressive = tf.concat(predictions_auto_regressive, axis=1)
predictions_prob_auto_regressive = tf.concat(predictions_prob_auto_regressive, axis=1)

tf.assert_equal(predictions_non_auto_regressive, predictions_auto_regressive)

In [39]:
# Batch predictions (var len)

text = ['Sachin Tendulkar is one of the finest', 
        'I love stars because']
# Batching using -1 is must
input_ids = tf.ragged.constant(tokenizer(text)['input_ids']).to_tensor(-1)
inputs = {}
inputs['input_ids'] = input_ids

seq_length = tf.shape(input_ids)[1]
batch_size = tf.shape(input_ids)[0]

inputs["all_cache_key"] = tf.zeros((12, batch_size, 12, seq_length, 64))
inputs["all_cache_value"] = tf.zeros((12, batch_size, 12, seq_length, 64))
inputs["past_length"] = tf.zeros(shape=(1, batch_size), dtype=tf.int32)
predictions_auto_regressive = []
predictions_prob_auto_regressive = []

past_lengths = []
for i in range(10):
    outputs = model(inputs)
    predicted_ids = tf.cast(tf.expand_dims(tf.argmax(outputs["last_token_logits"], axis=1), 1), tf.int32)
    
    if i == 0:
        masks = tf.cast(tf.not_equal(input_ids, -1), tf.float32)
        masks = tf.reshape(
            masks,
            (1, batch_size, 1, seq_length, 1),
        )
        outputs["all_cache_key"]   = outputs["all_cache_key"]   * masks
        outputs["all_cache_value"] = outputs["all_cache_value"] * masks
        
    inputs["input_ids"] = predicted_ids
    inputs["all_cache_key"] = outputs["all_cache_key"]
    inputs["all_cache_value"] = outputs["all_cache_value"]
    inputs["past_length"] = outputs["past_length"]
    past_lengths.append(inputs["past_length"])
    predictions_auto_regressive.append(predicted_ids)
    predictions_prob_auto_regressive.append(
        tf.expand_dims(tf.reduce_max(outputs["last_token_logits"], axis=1), 1)
    )
    
    
predictions_auto_regressive = tf.concat(predictions_auto_regressive, axis=1)
predictions_prob_auto_regressive = tf.concat(predictions_prob_auto_regressive, axis=1)
expected_ids = [[1938, 287, 262, 995, 13, 679, 318, 257, 845, 922],
                 [484, 821, 523, 881, 517, 621, 655, 257, 3491, 13]]
np.allclose(predictions_auto_regressive.numpy().tolist(), expected_ids)

In [ ]:
# Text generation using saved_model with TextDecoder

import tempfile
import shutil
from tf_transformers.text import TextDecoder

text = ['Sachin Tendulkar is one of the finest', 
        'I love stars because']

saved_model_dir = tempfile.mkdtemp()
model.save_as_serialize_module(saved_model_dir, overwrite=True)

loaded   = tf.saved_model.load(saved_model_dir)
decoder  = TextDecoder(
    model = loaded
)

# -1 batch
input_ids = tf.ragged.constant(tokenizer(text)['input_ids']).to_tensor(-1)
inputs = {}
inputs['input_ids'] = input_ids

decoder_results = decoder.decode(inputs, 
               mode='greedy', 
               max_iterations=10, 
               eos_id=-100)
expected_ids = [[1938, 287, 262, 995, 13, 679, 318, 257, 845, 922],
                 [484, 821, 523, 881, 517, 621, 655, 257, 3491, 13]]
np.allclose(decoder_results['predicted_ids'].numpy().tolist(), 
           expected_ids)

In [31]:
# Text generation using saved_model with TextDecoderSerializable

import tempfile
import shutil
from tf_transformers.text import TextDecoderSerializable

text = ['Sachin Tendulkar is one of the finest', 
        'I love stars because']

saved_model_dir = tempfile.mkdtemp()
model.save_as_serialize_module(saved_model_dir, overwrite=True)

loaded   = tf.saved_model.load(saved_model_dir)
decoder  = TextDecoderSerializable(
    model = model,
    max_iterations=10,
    mode="greedy",
    do_sample=False,
    eos_id=-100
)

# Save
decoder_model = decoder.get_model()
decoder_model.save_serialized(saved_model_dir, overwrite=True)

# Load
loaded_decoder   = tf.saved_model.load(saved_model_dir)
model_pb_decoder = loaded_decoder.signatures['serving_default']

# -1 batch
input_ids = tf.ragged.constant(tokenizer(text)['input_ids']).to_tensor(-1)
inputs = {}
inputs['input_ids'] = input_ids

decoder_results_serialized = model_pb_decoder(**inputs)
expected_ids = [[[1938, 287, 262, 995, 13, 679, 318, 257, 845, 922]],
               [[484, 821, 523, 881, 517, 621, 655, 257, 3491, 13]]]
np.allclose(decoder_results_serialized['predicted_ids'].numpy().tolist(), 
           expected_ids)